# Component 3: Preprocessing & Data PipelinesDefine preprocessing variants, augmentation, and build tf.data pipelines

In [ ]:
import tensorflow as tfimport pandas as pdimport numpy as npimport matplotlib.pyplot as pltimport osSEED = 42tf.random.set_seed(SEED)np.random.seed(SEED)IMG_SIZE = (224, 224)BATCH_SIZE = 32# Load manifeststrain_df = pd.read_csv('../outputs/train_manifest.csv')val_df = pd.read_csv('../outputs/val_manifest.csv')test_df = pd.read_csv('../outputs/test_manifest.csv')print(f'Train: {len(train_df)} images')print(f'Val:   {len(val_df)} images')print(f'Test:  {len(test_df)} images')print(f'Classes: {len(train_df["class_label"].unique())}')

## 3.1 Preprocessing Functions

In [ ]:
def preprocess_resize_rescale(filepath, label):    """Simple resize and rescale to [0,1]."""    img = tf.io.read_file(filepath)    img = tf.image.decode_jpeg(img, channels=3)    img = tf.image.resize(img, IMG_SIZE)    img = img / 255.0    return img, labeldef preprocess_resnet(filepath, label):    """ResNet50-specific preprocessing."""    img = tf.io.read_file(filepath)    img = tf.image.decode_jpeg(img, channels=3)    img = tf.image.resize(img, IMG_SIZE)    img = tf.keras.applications.resnet50.preprocess_input(img)    return img, labeldef preprocess_efficientnet(filepath, label):    """EfficientNet-specific preprocessing."""    img = tf.io.read_file(filepath)    img = tf.image.decode_jpeg(img, channels=3)    img = tf.image.resize(img, IMG_SIZE)    img = tf.keras.applications.efficientnet.preprocess_input(img)    return img, labeldef preprocess_densenet(filepath, label):    """DenseNet-specific preprocessing."""    img = tf.io.read_file(filepath)    img = tf.image.decode_jpeg(img, channels=3)    img = tf.image.resize(img, IMG_SIZE)    img = tf.keras.applications.densenet.preprocess_input(img)    return img, labelprint("✓ Preprocessing functions defined")

## 3.2 Data Augmentation

In [ ]:
# Define augmentation pipelineaugmentation = tf.keras.Sequential([    tf.keras.layers.RandomFlip('horizontal'),    tf.keras.layers.RandomRotation(0.2),    tf.keras.layers.RandomZoom(0.2),    tf.keras.layers.RandomContrast(0.2)], name='augmentation')print("✓ Augmentation pipeline created")print("  - RandomFlip (horizontal)")print("  - RandomRotation (±20%)")print("  - RandomZoom (±20%)")print("  - RandomContrast (±20%)")

## 3.3 Build tf.data Pipelines

In [ ]:
def build_dataset(dataframe, preprocess_fn, augment=False, shuffle=True, batch_size=BATCH_SIZE):    """Build optimized tf.data pipeline."""    filepaths = dataframe['filepath'].values    labels = dataframe['class_label'].values        # Create dataset    ds = tf.data.Dataset.from_tensor_slices((filepaths, labels))        # Preprocessing    ds = ds.map(preprocess_fn, num_parallel_calls=tf.data.AUTOTUNE)        # Cache before augmentation    ds = ds.cache()        # Augmentation (only for training)    if augment:        ds = ds.map(lambda x, y: (augmentation(x, training=True), y),                    num_parallel_calls=tf.data.AUTOTUNE)        # Shuffle    if shuffle:        ds = ds.shuffle(buffer_size=1000, seed=SEED)        # Batch and prefetch    ds = ds.batch(batch_size)    ds = ds.prefetch(tf.data.AUTOTUNE)        return dsprint("✓ Dataset building function defined")

## 3.4 Create Datasets (Basic Preprocessing)

In [ ]:
# Build datasets with simple resize+rescaletrain_ds = build_dataset(train_df, preprocess_resize_rescale, augment=True, shuffle=True)val_ds = build_dataset(val_df, preprocess_resize_rescale, augment=False, shuffle=False)test_ds = build_dataset(test_df, preprocess_resize_rescale, augment=False, shuffle=False)print("✓ Datasets created")print(f"  Train: {len(train_df)} images, {len(train_ds)} batches")print(f"  Val:   {len(val_df)} images, {len(val_ds)} batches")print(f"  Test:  {len(test_df)} images, {len(test_ds)} batches")

## 3.5 Test Pipeline

In [ ]:
# Test the pipelineprint("Testing pipeline with a sample batch...")for images, labels in train_ds.take(1):    print(f"\nBatch shape: {images.shape}")    print(f"Labels shape: {labels.shape}")    print(f"Image dtype: {images.dtype}")    print(f"Image range: [{images.numpy().min():.3f}, {images.numpy().max():.3f}]")    print(f"Labels: {labels.numpy()}")        # Visualize a few samples    fig, axes = plt.subplots(2, 4, figsize=(16, 8))    for i, ax in enumerate(axes.flatten()):        if i < images.shape[0]:            ax.imshow(images[i].numpy())            ax.set_title(f'Label: {labels[i].numpy()}')            ax.axis('off')    plt.suptitle('Sample Batch from Training Pipeline', fontsize=14, fontweight='bold')    plt.tight_layout()    plt.show()print("\n" + "="*60)print("✅ PREPROCESSING COMPLETE")print("="*60)print("\nDatasets are ready for model training!")print("\nAvailable preprocessing functions:")print("  - preprocess_resize_rescale (for Baseline CNN)")print("  - preprocess_resnet (for ResNet50)")print("  - preprocess_efficientnet (for EfficientNetB0)")print("  - preprocess_densenet (for DenseNet121)")